In [1]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import re
from time import sleep
import json

import aiohttp
import asyncio

In [2]:
category = {'Завтраки':'zavtraki', 
            'Закуски':'zakuski', 'Напитки':'napitki', 
            'Основные блюда':'osnovnye-blyuda', 'Паста и пицца':'pasta-picca', 
            'Салаты':'salaty', 
            'Соусы и маринады':'sousy-marinady', 'Супы':'supy', 
            'Выпечка и десерты':'vypechka-deserty'}


kuhni = {'Итальянская кухня':'italyanskaya-kuhnya', 'Грузинская кухня':'gruzinskaya-kuhnya', 
            'Китайская кухня':'kitayskaya-kuhnya', 'Французская кухня':'francuzskaya-kuhnya', 
            'Русская кухня':'russkaya-kuhnya', 'Японская кухня':'yaponskaya-kuhnya', 
            'Индийская кухня':'indiyskaya-kuhnya', 'Мексиканская кухня':'meksikanskaya-kuhnya', 
            'Армянская кухня':'armyanskaya-kuhnya', 'Американская кухня':'amerikanskaya-kuhnya', 
            'Испанская кухня':'ispanskaya-kuhnya', 'Немецкая кухня':'nemeckaya-kuhnya', 
            'Греческая кухня':'grecheskaya-kuhnya', 'Азербайджанская кухня':'azerbaydzhanskaya-kuhnya', 
            'Европейская кухня':'evropeyskaya-kuhnya', 'Еврейская кухня':'evreyskaya-kuhnya', 
            'Корейская кухня':'koreyskaya-kuhnya', 'Тайская кухня':'tayskaya-kuhnya', 
            'Паназиатская кухня':'panaziatskaya-kuhnya', 'Турецкая кухня':'tureckaya-kuhnya', 
            'Узбекская кухня':'uzbekskaya-kuhnya', 'Татарская кухня':'tatarskaya-kuhnya', 
            'Средиземноморская кухня':'sredizemnomorskaya-kuhnya', 'Арабская кухня':'arabskaya-kuhnya', 
            'Украинская кухня':'ukrainskaya-kuhnya', 'Польская кухня':'polskaya-kuhnya', 
            'Британская кухня':'britanskaya-kuhnya', 'Белорусская кухня':'belorusskaya-kuhnya', 
            'Норвежская кухня':'norvezhskaya-kuhnya', 'Шведская кухня':'shvedskaya-kuhnya', 
            'Марокканская кухня':'marokkanskaya-kuhnya', 'Болгарская кухня':'bolgarskaya-kuhnya', 
            'Австрийская кухня':'avstriyskaya-kuhnya', 'Австралийская кухня':'avstraliyskaya-kuhnya', 
            'Финская кухня':'finskaya-kuhnya', 'Сербская кухня':'serbskaya-kuhnya', 
            'Венгерская кухня':'vengerskaya-kuhnya', 'Вьетнамская кухня':'vetnamskaya-kuhnya', 
            'Молдавская кухня':'moldavskaya-kuhnya', 'Осетинская кухня':'osetinskaya-kuhnya', 
            'Канадская кухня':'canadian-cuisine', 'Алжирская кухня':'algerian-cuisine', 
            'Дагестанская кухня':'dagestan-cuisine', 'Албанская кухня':'albanian-cuisine', 
            'Мировая кухня':'world-cuisine', 'Туркменская кухня':'turkmenskaya-kuhnya', 
            'Афганская кухня':'afganskaya-kuhnya', 'Египетская кухня':'egipetskaya-kuhnya', 
            'Крымская кухня':'krymskaya-kuhnya', 
            'Африканская кухня':'afikanskaya-kuhnya', 'Филиппинская кухня':'filippinskaya-kuhnya', 
            'Шотландская кухня':'shotlandskaya-kuhnya', 'Сирийская кухня':'siriyskaya-kuhnya', 
            'Таджикская кухня':'tadzhikskaya-kuhnya', 'Португальская кухня':'portugalskaya-kuhnya', 
            'Латиноамериканская кухня':'latinoamerikanskaya-kuhnya', 'Латвийская кухня':'latviyskaya-kuhnya', 
            'Индонезийская кухня':'indoneziyskaya-kuhnya', 'Датская кухня':'datskaya-kuhnya', 
            'Бурятская кухня':'buryat-cuisine', 'Кавказская кухня':'caucasus-cuisine', 
            'Колумбийская кухня':'gastronomia-de-colombia', 'Креольская кухня':'louisiana-cuisine', 
            'Сицилийская кухня':'sicilian-cuisine', 'Советская кухня':'soviet-cuisine', 
            'Камбоджийская кухня':'kambodzhiyskaya-kuhnya', 
            'Казахская кухня':'kazahskaya-kuhnya', 'Чеченская кухня':'chechenskaya-kuhnya', 
            'Южно-индийская кухня':'yuzhno-indiyskaya-kuhnya', 
            'Скандинавская кухня':'skandinavskaya-kuhnya', 'Бразильская кухня':'brazilskaya-kuhnya', 
            'Кухня Вестероса':'vesterosskaya-kuhnya', 
            'Голландская кухня':'gollandskaya-kuhnya', 'Чешская кухня':'cheshskaya-kuhnya', 
            'Литовская кухня':'litovskaya-kuhnya', 'Ливанская кухня':'livanskaya-kuhnya', 
            'Кубинская кухня':'kubinskaya-kuhnya', 'Киргизская кухня':'kirgizskaya-kuhnya', 
            'Башкирская кухня':'bashkirskaya-kuhnya', 'Исландская кухня':'icelandic-cuisine', 
            'Швейцарская кухня':'swiss-cuisine', 'Абхазская кухня':'abkhaziancuisine', 
            'Хорватская кухня':'khorvatskaya-kuhnya', 
            'Румынская кухня':'rumynskaya-kuhnya', 'Персидская кухня':'persidskaya-kuhnya', 
            'Сингапурская кухня':'singapurskaya-kuhnya', 'Черкесская кухня':'cherkesskaya-kuhnya', 
            'Восточно-индийская кухня':'vostochno-indiyskaya-kuhnya', 'Одесская кухня':'odesskaya-kuhnya', 
            'Малайзийская кухня':'malaziyskaya-kuhnya', 'Ирландская кухня':'irlandskaya-kuhnya', 
            'Авторская кухня':'avtorskaya-kuhnya', 'Югославская кухня':'yugoslavskaya-kuhnya', 
            'Эстонская кухня':'estonskaya-kuhnya', 'Карибская кухня':'karibskaya-kuhnya', 
            'Бельгийская кухня':'belgiyskaya-kuhnya', 'Аргентинская кухня':'argentinskaya-kuhnya'}



menu = {'Безглютеновая диета':'bezglyutenovaya-dieta', 'Вегетарианская еда':'vegetarianskaya-eda', 
            'Веганская еда':'veganskaya-eda', 'Безлактозная диета':'bezlaktoznaya-dieta', 
            'Детское меню':'detskoe-menyu', 'Низкокалорийная еда':'nizkokaloriynaya-eda', 
            'Постная еда':'postnaya-eda', 'Меню при диабете':'menu-pri-diabete'}

In [3]:
for el in category.keys():
    print(el)

Завтраки
Закуски
Напитки
Основные блюда
Паста и пицца
Салаты
Соусы и маринады
Супы
Выпечка и десерты


In [37]:
# #Не обязательно запускать
# S = 0
# for el_kuhni in kuhni:
#     for el_category in category:
#         print(el_kuhni, kuhni[el_kuhni])
#         print(el_category, category[el_category])
#         url  = 'https://eda.ru/recepty/' + kuhni[el_kuhni] + '/' + category[el_category]
#         r = requests.get(url)
#         break;
#         soup = BeautifulSoup(r.text)
#         # # print(soup)
#         # tree = html.fromstring(soup)
#         num_recipes = soup.find('span', {'class': 'emotion-1ad0u8b'}).text 
#     #     print(num_recipes)
#         print(re.search('\d+', num_recipes)[0])
#         if int(re.search('\d+', num_recipes)[0]) >= 10000:
#             print('!!!')
#         S += int(re.search('\d+', num_recipes)[0])
#     break;
# S

In [24]:
payload = {'key1': 'value1', 'key2': 'value2'}
r = requests.get('https://httpbin.org/get', params=payload)

In [25]:
r.url

'https://httpbin.org/get?key1=value1&key2=value2'

In [38]:
async with aiohttp.ClientSession() as session:
    async with session.get('http://httpbin.org/get') as resp:
        # print(resp.status)
        print(await resp.text())


{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "Python/3.10 aiohttp/3.8.3", 
    "X-Amzn-Trace-Id": "Root=1-63c5483d-3bed3a9429fc40a0059f61ab"
  }, 
  "origin": "95.189.197.56", 
  "url": "http://httpbin.org/get"
}



In [35]:
async def async_generator():
    for i in range(10):
        # block to simulate doing work
        await asyncio.sleep(1)
        # yield the result
        yield i

In [36]:
async for x in async_generator():
    print(x)

0
1
2
3
4
5
6
7
8
9


In [15]:

S

0

In [43]:
# url  = 'https://eda.ru/recepty/vypechka-deserty/brauni-brownie-20955'
def get_data(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    # Тут находим теги
    recipe_tags = soup.find('ul', {'class': 'emotion-1bqzczv'})
    menu_tag = 'Еда без ограничений'
    for child in recipe_tags.children:
#         print(child.text)
        if child.text in menu:
            menu_tag = child.text
    # Тут находим название блюда
    recipe_title = soup.find('h1', {'class': 'emotion-gl52ge'})
    title = recipe_title.text 
#     print(title)
    # Тут находим описание блюда
    recipe_descr = soup.find('span', {'class': 'emotion-1x1q7i2'})
    descripion = recipe_descr.text if recipe_descr is not None else ''
#     print(descripion)
    return {'title': title, 'menu': menu_tag, 'descripion': descripion}


In [44]:
url  = 'https://eda.ru/recepty/vypechka-deserty/brauni-brownie-20955'
get_data(url)

{'title': 'Брауни (brownie)',
 'menu': 'Еда без ограничений',
 'descripion': 'Один из\xa0самых популярных десертов в\xa0мире\xa0— брауни\xa0— был придуман в\xa01893 году на\xa0кухне легендарного отеля Palmer House в\xa0Чикаго. Этот пирог там пекут до\xa0сих пор по\xa0оригинальному рецепту, покрывая сверху абрикосовой глазурью. В\xa0домашней версии, впрочем, у\xa0брауни получается такая изумительная сахарная корочка, что глазировать ее\xa0было\xa0бы преступлением.\nУ\xa0традиционных шоколадных брауни ванильный аромат, хрустящая корочка и\xa0влажная серединка. В\xa0торт также добавляют грецкие орехи или фисташки, а\xa0еще клюкву.'}

In [41]:
# url  = 'https://eda.ru/recepty?page=714'
def get_sites(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    
    all_sites = []
    
    list_recipes = soup.find('div', {'class': 'emotion-7ote3x'})
    for child in list_recipes.children:
        for recipe in child.children:
            if recipe.find('a', {'class': 'emotion-12sjte8'}) is not None:
#                 print(recipe.find('a', {'class': 'emotion-12sjte8'}).get('href'), '\n\n\n')
                all_sites.append('https://eda.ru' + recipe.find('a', {'class': 'emotion-12sjte8'}).get('href'))

    list_recipes = soup.find('div', {'class': 'emotion-1ywwzp6'})
    for child in list_recipes.children:
        for recipe in child.children:
            if recipe.find('a', {'class': 'emotion-12sjte8'}) is not None:
#                 print(recipe.find('a', {'class': 'emotion-12sjte8'}).get('href'), '\n\n\n')
                all_sites.append('https://eda.ru' + recipe.find('a', {'class': 'emotion-12sjte8'}).get('href'))
    return all_sites


In [42]:
url  = 'https://eda.ru/recepty?page=714'
get_sites(url)

['https://eda.ru/recepty/sousy-marinady/majonez-domashnij-bez-jaic-s-jablochnim-uksusom-35302',
 'https://eda.ru/recepty/vypechka-deserty/hachapuri-iz-tykvy-137478',
 'https://eda.ru/recepty/zakuski/karpachcho-iz-cukini-14422',
 'https://eda.ru/recepty/vypechka-deserty/pechene-alenka-v-pelenke--151676',
 'https://eda.ru/recepty/salaty/salat-iz-jajca-i-zelenogo-luka-52795',
 'https://eda.ru/recepty/salaty/salat-so-shhavelem-44591',
 'https://eda.ru/recepty/supy/groningenskij-gorchichnij-sup-17560',
 'https://eda.ru/recepty/osnovnye-blyuda/sote-iz-anchousov-s-brokkoli-19194',
 'https://eda.ru/recepty/rizotto/rizotto-po-milanski-s-shafranom-33950',
 'https://eda.ru/recepty/osnovnye-blyuda/ribaj-stejk-s-kartofelnim-pjure-i-slivochno-gribnim-sousom-iz-smorchkov-46480',
 'https://eda.ru/recepty/vypechka-deserty/pesochnoe-pechene-s-makom-24304',
 'https://eda.ru/recepty/salaty/rukola-s-zharenymi-krevetkami-140239',
 'https://eda.ru/recepty/supy/ramen-46326',
 'https://eda.ru/recepty/supy/tadz

In [211]:
el_category = 'Выпечка и десерты'

for el_kuhni in kuhni:
#     for el_category in category:
    print(el_kuhni, kuhni[el_kuhni])
#     print(el_category, category[el_category])
    url  = 'https://eda.ru/recepty/' + kuhni[el_kuhni] + '/' + category[el_category]
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    num_recipes = soup.find('span', {'class': 'emotion-1ad0u8b'}).text
    num_recipes = int(re.search('\d+', num_recipes)[0])

    on_one_page = 14
    num_pages = (num_recipes - 1)//on_one_page + 1
    max_page = min(714, num_pages)
    for i in range(1, max_page+1):
        url_in_loop = url + '?page=' + str(i)
        sites = get_sites(url_in_loop)
        for j in sites:
            #TO DO
            print(j)
            data = get_data(j)
            print(data)
            data_dict = {'title':data[0], 'category': el_category,
                        'cuisine': el_kuhni, 'menu':data[1], 
                         'description': data[2]}
            with open("data_file.json", "a+") as write_file:
                json.dump(data_dict, write_file, ensure_ascii=False)
                write_file.write("\n")

            sleep(1.5)

#         break


Итальянская кухня italyanskaya-kuhnya
Завтраки zavtraki
https://eda.ru/recepty/zavtraki/fruktovij-zavtrak-s-orehami-42320
('Фруктовый завтрак с\xa0орехами', 'Еда без ограничений', '')
https://eda.ru/recepty/zavtraki/legkiyzavtrak-50558
('Легкий завтрак', 'Еда без ограничений', 'Помидоры, начиненные яйцом и\xa0моцареллой. Помидоры должны быть спелые, но\xa0крепкие, чтобы после пребывания в\xa0духовке они смогли сохранить форму.')
https://eda.ru/recepty/zavtraki/yogurto-tvorozhnyy-zavtrak-s-myatoy-91440
('Йогурто-творожный завтрак с\xa0мятой', 'Еда без ограничений', '')
https://eda.ru/recepty/zavtraki/frittata-s-brokkoli-sladkim-percem-17602
('Фриттата с\xa0брокколи и\xa0сладким перцем', 'Еда без ограничений', 'Фриттата с\xa0брокколи и\xa0сладким перцем\xa0— классический итальянский омлет. Главный ингредиент здесь не\xa0яйца, а\xa0крупно нарезанные овощи, поэтому их\xa0должно быть много. Яйца только фиксируют конструкцию. Получается объемное, рельефное и\xa0красивое блюдо. Фриттата снача

KeyboardInterrupt: 

In [138]:
714*14

9996

In [15]:
with open("data_file.json", "r+", encoding='utf-8') as write_file:
    counter = 0
    for line in write_file:
        data_ = json.loads(line)
#         if not (data_['category'] == 'Завтраки' and data_['cuisine'] == 'Итальянская кухня'):
#         print(data_['category'], data_['cuisine'])
        counter += 1
    print(counter)           

Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
Завтраки Итальянская кухня
З